<a href="https://colab.research.google.com/github/chhavikirtani2000/Math-Question-Anwering/blob/main/Math_Question_Answering_Machine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import csv
import random
import re
import numpy as np
from nltk.stem.snowball import SnowballStemmer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.svm import SVC
from keras.layers import Input, Dense, Embedding, Conv2D, MaxPool2D
from keras.layers import Reshape, Flatten, Dropout, Concatenate
from keras.preprocessing.sequence import pad_sequences
from keras.callbacks import ModelCheckpoint
from keras.optimizers import Adam
from keras.models import Model
from keras.utils import np_utils
#questions, operations, answers
def regex_rule(X_test,op_predict,ans_test):
    ans_predict=[]
    for i in range(len(op_predict)):
      print(op_predict[i])
      print(X_test[i])
      nums=[]
      flag=0
      if op_predict[i]=='Addition':
        #nums=[]
        try:
          target = re.search("[Hh]ow (many|much)( more | )(\w+)",X_test[i]).group(3)
          X_test_mod = re.sub(target,'p',X_test[i])
          print("1: ",target)
          print("2: ",X_test_mod)
          try:
            targets = re.finditer("(\d+)( | more )(p|of them|than)", X_test_mod)
            print("3: ",targets)
            for t in targets:
                print("4: ",t)
                print("5: ",t.group(1))
                nums.append(t.group(1))
          except:pass
          flag=1
        except: pass
      elif op_predict[i]=='Subtraction':
        #nums=[]
        try:
          target = re.search("[Hh]ow (many|much)( more | )(\w+)",X_test[i]).group(3)
          X_test_mod = re.sub(target,'p',X_test[i])
          print("1: ",target)
          print("2: ",X_test_mod)
          try:
            targets = re.finditer("(\d+)( | less )(p|of them|than)", X_test_mod)
            print(targets)
            for t in targets:
                print("4: ",t)
                print("5: ",t.group(1))
                nums.append(t.group(1))
          except:pass
          flag=1
        except: pass
      elif op_predict[i]=='Multiplication':
        #nums=[]
        try:
          target = re.search("[Hh]ow (many|much)( more | )(\w+)",X_test[i]).group(3)
          X_test_mod = re.sub(target,'p',X_test[i])
          print("1: ",target)
          print("2: ",X_test_mod)
          try:
            targets = re.finditer("(\d+)( | of )(p|of them|than|at)", X_test_mod)
            print(targets)
            for t in targets:
                print("4: ",t)
                print("5: ",t.group(1))
                nums.append(t.group(1))
          except:pass
          '''if len(nums)==0:
            targets = re.finditer("( | of )(p|of them|than|at)(.*| )*(\d+)", X_test_mod)
            for t in targets:
                nums.append(t.group(1))'''
          flag=1
        except: pass
      elif op_predict[i]=='CommonDiv':
        #nums=[]
        try:
          target = re.search("[Hh]ow (many|much)( more | )(\w+)",X_test[i]).group(3)
          X_test_mod = re.sub(target,'p',X_test[i])
          print("1: ",target)
          print("2: ",X_test_mod)
          try:
            #targets = re.finditer("(\d+)( | of )(p|of them|than|at)", X_test_mod)
            targets1 = re.compile("(p|of them|than|it)( )(\b.*\b)( )(\d+)( )(.*)(\.|,)", X_test_mod)
            #.compile(r"\bis\b")
            targets = targets1.findall(y)[0]
            print("3: ",targets)
            for t in targets:
                print("4: ",t)
                print("5: ",t.group(1))
                nums.append(t.group(1))
          except:pass
          '''if len(nums)==0:
            targets = re.finditer("( | of )(p|of them|than|at)(.*| )*(\d+)", X_test_mod)
            for t in targets:
                nums.append(t.group(1))'''
          flag=1
        except: pass
        
      '''if flag==1:
        for t in targets:
              nums.append(t.group(1))'''
      if op_predict[i]=='Addition':
        if len(nums)>1:
          #print("here1")
          ans_predict.append(float(nums[0])+float(nums[1]))
        else:
          ans_predict.append('Cannot answer!')
      elif op_predict[i]=='Subtraction':
        if len(nums)>1 and float(nums[1])!=0:
          #print("here2")
          ans_predict.append(abs(float(nums[0])-float(nums[1])))
        else:
          ans_predict.append('Cannot answer!')
      elif op_predict[i]=='Multiplication':
        if len(nums)>1:
          #print("here3")
          ans_predict.append(float(nums[0])*float(nums[1]))
        else:
          ans_predict.append('Cannot answer!')
      elif op_predict[i]=='CommonDiv':
        if len(nums)>1 and float(nums[1])!=0.0:
          #print("here4")
          ans_predict.append(float(nums[0])/float(nums[1]))
        else:
          ans_predict.append('Cannot answer!')

    total_attempted = 0
    correct_ans = 0
    for i in range(len(ans_predict)):
        if ans_predict[i]=='Cannot answer!':
          continue
        elif ans_predict[i]!='Cannot answer!':
            total_attempted+=1
            if op_predict[i]!='Subtraction':
              if float(ans_predict[i])==float(ans_test[i]):
                correct_ans+=1
            else:
              if abs(float(ans_predict[i]))==abs(float(ans_test[i])):
                correct_ans+=1

    print("Total Attempted, Total Correct")
    print(total_attempted,correct_ans)
    print("Accuracy:")
    print(float(correct_ans/total_attempted))
''' To run this, just upload math_dataset.csv '''
def split_train_val(X_train, op_train):
  samples = list(zip(X_train,op_train))  #combine the representation
  random.seed(13)
  random.shuffle(samples) #random shuffling
  length=len(samples)
  k=int(0.8*length)
  X_train, op_train = zip(*samples[80:220])
  X_test, op_test= zip(*samples[:80])
  return np.array(X_train),np.array(op_train),np.array(X_test),np.array(op_test)
def shuffle_list(*ls):
    l =list(zip(*ls))
    random.shuffle(l)
    return zip(*l)

def build_vocab(docs):
    vocab = []
    for doc in docs:
        vocab += doc.split()
    vocab = list(set(vocab))
    w_to_i = {w:i for i,w in enumerate(vocab,1)} #enumerate from 1 to avoid confusion with zero padding
    i_to_w = {i:w for i,w in enumerate(vocab,1)}

    return w_to_i, i_to_w

def one_hot(docs,w_to_i):
    embeddings = []
    for doc in docs:
        embeddings.append([w_to_i[word] for word in doc.split() if word in w_to_i])

    return embeddings


def predict_operator(X_train, op_train, X_test, op_test):
    one_hot_dict, inv_one_hot_dict = build_vocab(X_train)
    vocabulary_size = len(one_hot_dict)+1
    X_train_embedding = one_hot(X_train,one_hot_dict)
    X_test_embedding = one_hot(X_test,one_hot_dict)
    category_num = len(set(op_train))
    categories = list(set(op_train))
    o_to_i = {o:i for i,o in enumerate(categories)}
    i_to_o = {i:o for i,o in enumerate(categories)}
    y_train_numeric = []
    y_test_numeric = []
    for label in op_train:
        y_train_numeric.append(o_to_i[label])
    for label in op_test:
        y_test_numeric.append(o_to_i[label])

    y_train_distribution = np_utils.to_categorical(y_train_numeric, category_num)
    y_test_distribution = np_utils.to_categorical(y_test_numeric, category_num)

    # pad documents to max length
    max_length = 100
    X_train_embedding_padded = pad_sequences(X_train_embedding, maxlen=max_length, padding='post')
    X_test_embedding_padded = pad_sequences(X_test_embedding, maxlen=max_length, padding='post')


    X_shuffled, y_shuffled = shuffle_list(X_train_embedding_padded, y_train_distribution)
    length = len(X_shuffled)
    X_train_onehot = np.array(X_shuffled[:int(0.8*length)])
    X_dev_onehot = np.array(X_shuffled[int(0.8*length):])
    y_train_distribution = np.array(y_shuffled[:int(0.8*length)])
    y_dev_distribution = np.array(y_shuffled[int(0.8*length):])
    embedding_dim = 256
    filter_sizes = [3, 4, 5]
    num_filters = 64
    drop = 0.2

    epochs = 10
    batch_size = 128

    print("Creating Model...")
    inputs = Input(shape=(max_length,), dtype='int32')
    embedding = Embedding(input_dim=vocabulary_size, output_dim=embedding_dim, input_length=max_length)(inputs)
    reshape = Reshape((max_length, embedding_dim, 1))(embedding)

    conv_0 = Conv2D(num_filters, kernel_size=(filter_sizes[0], embedding_dim), padding='valid', kernel_initializer='normal',
                activation='relu')(reshape)
    conv_1 = Conv2D(num_filters, kernel_size=(filter_sizes[1], embedding_dim), padding='valid', kernel_initializer='normal',
                activation='relu')(reshape)
    conv_2 = Conv2D(num_filters, kernel_size=(filter_sizes[2], embedding_dim), padding='valid', kernel_initializer='normal',
                activation='relu')(reshape)

    maxpool_0 = MaxPool2D(pool_size=(max_length - filter_sizes[0] + 1, 1), strides=(1, 1), padding='valid')(conv_0)
    maxpool_1 = MaxPool2D(pool_size=(max_length - filter_sizes[1] + 1, 1), strides=(1, 1), padding='valid')(conv_1)
    maxpool_2 = MaxPool2D(pool_size=(max_length - filter_sizes[2] + 1, 1), strides=(1, 1), padding='valid')(conv_2)

    concatenated_tensor = Concatenate(axis=1)([maxpool_0, maxpool_1, maxpool_2])
    flatten = Flatten()(concatenated_tensor)
    dropout = Dropout(drop)(flatten)
    output = Dense(units=category_num, activation='softmax')(dropout)

    model = Model(inputs=inputs, outputs=output)

    checkpoint = ModelCheckpoint('CNN.hdf5', monitor='val_loss', verbose=0, save_best_only=True, mode='auto')
    adam = Adam(lr=2e-3, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)

    model.compile(optimizer=adam, loss='categorical_crossentropy', metrics=['accuracy'])
    #print(model.summary())
    print("Training Model...")
    model.fit(X_train_onehot, y_train_distribution, batch_size=batch_size, epochs=epochs, verbose=0, callbacks=[checkpoint],
         validation_data=(X_dev_onehot, y_dev_distribution))

    model.load_weights("CNN.hdf5")
    y_predict = model.predict(X_test_embedding_padded)
    y_predict = [i_to_o[np.argmax(i)] for i in y_predict]
    #print(y_predict)
    print("Operation acc: {}".format(model.evaluate(X_test_embedding_padded,y_test_distribution,verbose=0)[1]))
    return y_predict

questions = []
operators = []
answers = []
equations = []
num=0
with open('dataset.csv') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    for row in csv_reader:
          if num==0:
            num+=1
            continue
          else:
            questions.append(row[0])
            operators.append(row[1])
            answers.append(row[2])
            equations.append(row[3])
            num+=1
def convert(l): 
    return tuple(l) 
print("Total number of samples: ",num)
samples = list(zip(questions,operators,answers,equations))  #combine the representation
#print(samples)
import nltk
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer 
lemmatizer = WordNetLemmatizer()
for i in range(len(samples)):
  #print(list(samples[i]))
  l=list(samples[i])
  s=""
  for k in (list(samples[i])[0]).split():
    #print(k)
    #print("mod K: ",lemmatizer.lemmatize(k))
    s=s+" "+lemmatizer.lemmatize(k)
  #s=lemmatizer.lemmatize(list(samples[i])[0])
  #print(s[1:])
  l[0] = s[1:]
  #list=l
  samples[i]=convert(l)
#random.seed(13)
#random.shuffle(samples) #random shuffling
#print(samples)
''' X_train/test = questions, op_train/test = operators, ans_train/test = answers, eq_train/test = equations '''
print("Dataset is divided by 70-30 train-test % ")
X_train, op_train, ans_train, eq_train = zip(*samples[60:])
X_test, op_test, ans_test, eq_test = zip(*samples[:60])
#X_train = lemmatizer.lemmatize(X_train)
#X_test = lemmatizer.lemmatize(X_test)
op_predict = predict_operator(X_train, op_train, X_test, op_test)
print(op_predict)
regex_rule(X_test,op_predict,ans_test)
'''
target = re.search("[Hh]ow (many|much)( more | )(\w+)",X_test[i]).group(3)
X_test_mod = re.sub(target,'p',X_test[i])
print("1: ",target)
print("2: ",X_test_mod)
try:
  targets = re.finditer("(\d+)( | more )(p|of them|than|of)", X_test_mod)        (p|of them|than|it)( )(\b.*\b)( )(\d+)( )(.*)(\.)
  print("3: ",targets)
  for t in targets:
      print("4: ",t)
      print("5: ",t.group(1)) 
'''

Total number of samples:  959
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
Dataset is divided by 70-30 train-test % 
Creating Model...
Training Model...
Operation acc: 1.0
['Subtraction', 'Addition', 'Subtraction', 'Multiplication', 'Addition', 'Multiplication', 'Multiplication', 'Subtraction', 'Subtraction', 'Addition', 'Addition', 'Addition', 'Addition', 'CommonDiv', 'Multiplication', 'CommonDiv', 'Subtraction', 'CommonDiv', 'Subtraction', 'CommonDiv', 'CommonDiv', 'Multiplication', 'Addition', 'Subtraction', 'Multiplication', 'Addition', 'Addition', 'Multiplication', 'Addition', 'Addition', 'Subtraction', 'Subtraction', 'Subtraction', 'Addition', 'Multiplication', 'Subtraction', 'Subtraction', 'Addition', 'Subtraction', 'Addition', 'Addition', 'CommonDiv', 'Multiplication', 'Subtraction', 'CommonDiv', 'Subtraction', 'Subtraction', 'Addition', 'Subtraction', 'Subtraction', 'Subtraction', 'CommonDiv', 'Addition', 'S

'\ntarget = re.search("[Hh]ow (many|much)( more | )(\\w+)",X_test[i]).group(3)\nX_test_mod = re.sub(target,\'p\',X_test[i])\nprint("1: ",target)\nprint("2: ",X_test_mod)\ntry:\n  targets = re.finditer("(\\d+)( | more )(p|of them|than|of)", X_test_mod)        (p|of them|than|it)( )(\x08.*\x08)( )(\\d+)( )(.*)(\\.)\n  print("3: ",targets)\n  for t in targets:\n      print("4: ",t)\n      print("5: ",t.group(1)) \n'

In [ ]:
re.sub("book",'p',"there are some books there")
# do tokenization on sentences

'there are some ps there'

In [ ]:
X = "Horry take away 5 shelf . Lori ha 30 marble . If she share her marble among 5 friend , how many marble doe each friend get ?"

target = re.search("[Hh]ow (many|much)( more | )(\w+)",X).group(3)
X_test_mod = re.sub(target,'p',X)
print("1: ",target)
print("2: ",X_test_mod)
try:
  #targets = re.finditer("(\d+)( | of )(p|of them|than|at)", X_test_mod)
  #targets1 = re.compile("(p|of them|than|it)( )(\b.*\b)( )(\d+)( )(.*)(\.|,)", X_test_mod)
  #.compile(r"\bis\b")
  #print(targets1)
  #print(targets1.findall(X_test_mod))
  #targets = targets1.findall(X_test_mod)[0]
  targets = re.search("(p|of them|than|it)( )(.*)( )(\d+)( )(.*)(\.|,)",X_test_mod)
  print("3: ",targets)
  print("4: ",targets.group(0))
  t=targets.group(0)
  targets1 = re.search("(\d+)( )(\w+)",t)
  print(targets1)
  t1=targets1.group(0).split()
  num1=t1[0]
  wrd=t1[1]
  
  #print(targets.group(2))
  #nums.append(targets.group(1))
except: print("hi")

1:  marble
2:  Horry take away 5 shelf . Lori ha 30 p . If she share her p among 5 friend , how many p doe each friend get ?
3:  <_sre.SRE_Match object; span=(37, 76), match='p . If she share her p among 5 friend ,'>
4:  p . If she share her p among 5 friend ,
<_sre.SRE_Match object; span=(29, 37), match='5 friend'>
